<a href="https://colab.research.google.com/github/ryanskytree/Kgdc_Process/blob/main/KGDC_choose_K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch_geometric
import sys
sys.path.append("/content/drive/MyDrive/KorderGraphDiffusionConv/")
import GDC

In [ ]:
import argparse
import os.path as osp
import time

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.logging import init_wandb, log
from torch_geometric.nn import GCNConv


dataset='Cora'
lr=0.01
hidden_channels=16
epochs=1000
use_gdc='Use GDC'
wandb='Track experiment'


if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

init_wandb(
    name=f'GCN-{dataset}',
    lr=lr,
    epochs=epochs,
    hidden_channels=hidden_channels,
    device=device,
)

dataset = Planetoid(root='/tmp/cora', name='Cora', transform=T.NormalizeFeatures())


In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index, edge_weight=None):
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv1(x, edge_index, edge_weight).relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return x
model = GCN(
    in_channels     = dataset.num_features,
    hidden_channels = hidden_channels,
    out_channels    = dataset.num_classes,
).to(device)

optimizer = torch.optim.Adam([
    dict(params=model.conv1.parameters(), weight_decay=5e-4),
    dict(params=model.conv2.parameters(), weight_decay=5e-5)
], lr)  # Only perform weight-decay on first convolution.

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.edge_attr)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test():
    model.eval()
    pred = model(data.x, data.edge_index, data.edge_attr).argmax(dim=-1)
    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        accs.append(int((pred[mask] == data.y[mask]).sum()) / int(mask.sum()))
    return accs

In [ ]:
import GDC
choose_k_list=[]
for choose_k in range(1,20):
    transform = GDC.GDC(
        diffusion_kwargs=dict(method='korder', alpha=0.05,t=5),
        kinput = choose_k,
        )
    data = transform(dataset[0].to(device))
    best_val_acc = test_acc = 0
    times = []
    for epoch in range(1, epochs + 1):
        start = time.time()
        loss = train()
        train_acc, val_acc, tmp_test_acc = test()
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            test_acc = tmp_test_acc
        if epoch%100==0:
            log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        times.append(time.time() - start)
    print(f'Median time per epoch: {torch.tensor(times).median():.4f}s')
    choose_k_list.append(test_acc)
choose_k_list = np.array(choose_k_list)
print(np.max(choose_k_list))